<a href="https://colab.research.google.com/github/kurtlee1984/Computer_vision_project/blob/main/train_yolov4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import os
os.listdir('drive/My Drive')

['Colab Notebooks', 'Origin data', 'yolov4']

**1) Chang path to darknet**


In [ ]:
%cd /content/drive/My Drive/yolov4/darknet

/content/drive/My Drive/yolov4/darknet


**2) Compile Darknet using Nvidia GPU**

In [ ]:
#change makefile to have GPU and OPENCV enabled
#%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variable ‘it_tb_res’ [-Wunused-variable]
     int it_tb_

**3) Create train.txt and test.txt**

In [ ]:
import glob
import pandas as pd
import numpy as np
json_file_pathlist=glob.glob("data/training_dataset/*/*/*.jpg")
print(json_file_pathlist)

['data/training_dataset/Normal/origin/dac6b76fc.jpg', 'data/training_dataset/Normal/origin/dacaf5c55.jpg', 'data/training_dataset/Normal/origin/dae78abf2.jpg', 'data/training_dataset/Normal/origin/dae984d6a.jpg', 'data/training_dataset/Normal/origin/dafa14917.jpg', 'data/training_dataset/Normal/origin/db0fdecd3.jpg', 'data/training_dataset/Normal/origin/db1af9327.jpg', 'data/training_dataset/Normal/origin/db3eb3f0c.jpg', 'data/training_dataset/Normal/origin/dbcfa06bd.jpg', 'data/training_dataset/Normal/origin/dbd433d29.jpg', 'data/training_dataset/Normal/origin/dbe68e1c1.jpg', 'data/training_dataset/Normal/origin/dbfd38b38.jpg', 'data/training_dataset/Normal/origin/dc11d49a0.jpg', 'data/training_dataset/Normal/origin/dc1406483.jpg', 'data/training_dataset/Normal/origin/dc1c9be0d.jpg', 'data/training_dataset/Normal/origin/dc1e97cee.jpg', 'data/training_dataset/Normal/origin/dc2c73c4b.jpg', 'data/training_dataset/Normal/origin/dc3b08dc0.jpg', 'data/training_dataset/Normal/origin/dc50efc2

In [ ]:
len(json_file_pathlist)

In [ ]:
df=pd.DataFrame({"image_path":json_file_pathlist})
df.head()

,image_path
0,data/training_dataset/Normal/origin/dac6b76fc.jpg
1,data/training_dataset/Normal/origin/dacaf5c55.jpg
2,data/training_dataset/Normal/origin/dae78abf2.jpg
3,data/training_dataset/Normal/origin/dae984d6a.jpg
4,data/training_dataset/Normal/origin/dafa14917.jpg


In [ ]:
np.random.seed(5)
shuffle_index=np.random.permutation(df.shape[0])

In [ ]:
training_ratio=int(len(shuffle_index)*0.9)
train_index=shuffle_index[0:training_ratio]
test_index=shuffle_index[training_ratio::]

In [ ]:
training_data=df.iloc[train_index,:]
test_data=df.iloc[test_index,:]

In [ ]:
print(training_data)
print(test_index)

                                              image_path
5963   data/training_dataset/Flip/0/Flip_0_82344bf14.jpg
7044   data/training_dataset/Flip/1/Flip_1_e9491d2d6.jpg
7007   data/training_dataset/Flip/1/Flip_1_e8e90e8db.jpg
11180  data/training_dataset/Flip/-1/Flip_-1_a7605a14...
10718  data/training_dataset/Flip/-1/Flip_-1_84e6957a...
...                                                  ...
8992   data/training_dataset/Flip/1/Flip_1_695ae4a12.jpg
6818   data/training_dataset/Flip/1/Flip_1_d9c104dce.jpg
12016  data/training_dataset/Flip/-1/Flip_-1_4dc07d36...
1927   data/training_dataset/Normal/origin/8a7d9b4b0.jpg
10107  data/training_dataset/Flip/-1/Flip_-1_d5cc960f...

[12052 rows x 1 columns]
[1889 6739 7782 ... 4079 2254 2915]


In [ ]:
training_data.to_csv("data/train.txt",index=None,header=None)
test_data.to_csv("data/test.txt",index=None,header=None)

**4) Train model**

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg yolov4.conv.137 -dont_show

In [ ]:
!./darknet detector train data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights -dont_show